<a href="https://colab.research.google.com/github/Av01/Reinforcement-Learning/blob/master/Value_and_Policy_iteration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
class Game:
  def __init__(self,grid_size,slippery = 0):
    self.grid_size = grid_size
    self.slippery = slippery
    
  def take_action(self,pos,a):
    new_pos = [0,0]
    if np.random.random() < self.slippery:
      a = np.random.randint(0,self.grid_size-1)
    if a == 0: #up
      new_pos[0] = max(pos[0] - 1,0)
      new_pos[1] = pos[1]
    if a == 1: #down
      new_pos[0] = min(self.grid_size-1,pos[0] + 1)
      new_pos[1] = pos[1]
    if a == 2: #right
      new_pos[1] = min(self.grid_size-1,pos[1] + 1)
      new_pos[0] = pos[0]
    if a == 3: #left
      new_pos[1] = max(pos[1] - 1,0)
      new_pos[0] = pos[0]
    if new_pos[0] == self.grid_size-1 and new_pos[1] == self.grid_size-1:
      return new_pos,1,True
    elif new_pos[0] == self.grid_size-1 and new_pos[1] == 0:
      return new_pos,-1,True
    else:
      return new_pos,0,False
    
  def is_end_state(self,pos):
    if pos[0] == self.grid_size-1 and pos[1] == self.grid_size-1:
      return True
    elif pos[0] == self.grid_size-1 and pos[1] == 0:
      return True
    else:
      return False

In [0]:
grid_size = 4
n_states = grid_size * grid_size
n_action = 4

***Value Iteration***

In [131]:
import numpy as np
value_fun = np.zeros((grid_size,grid_size))
policy = np.zeros((grid_size,grid_size))
game = Game(grid_size,0.2)

episodes = 100
gamma = 0.5

for i in range(episodes):
  v = np.zeros((grid_size,grid_size))
  v[grid_size-1,grid_size-1] = 1
  v[grid_size-1,0] = -1
  for r in range(grid_size):
    for c in range(grid_size):
      pos = [r,c]
      if game.is_end_state(pos):
        continue
      max_v = -1
      for a in range(n_action):
        new_pos,rew,d = game.take_action(pos,a)
        if not d:
          max_v = max(rew + gamma*value_fun[new_pos[0],new_pos[1]],max_v)
        else:
          max_v = max(rew,max_v)
        #print(r,c,max_v)
      v[r,c] = max_v
  value_fun[:,:] = v[:,:]
  print(value_fun,"Iteration ",i)

[[ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  1.]
 [-1.  0.  1.  1.]] Iteration  0
[[ 0.   0.   0.   0. ]
 [ 0.   0.   0.   0.5]
 [ 0.   0.   0.5  1. ]
 [-1.   0.5  0.5  1. ]] Iteration  1
[[ 0.    0.    0.    0.25]
 [ 0.    0.    0.25  0.25]
 [ 0.    0.25  0.5   1.  ]
 [-1.    0.25  1.    1.  ]] Iteration  2
[[ 0.     0.     0.125  0.125]
 [ 0.     0.125  0.125  0.125]
 [ 0.125  0.25   0.5    0.5  ]
 [-1.     0.5    1.     1.   ]] Iteration  3
[[ 0.      0.0625  0.0625  0.0625]
 [ 0.0625  0.0625  0.0625  0.0625]
 [ 0.125   0.25    0.5     0.25  ]
 [-1.      0.5     1.      1.    ]] Iteration  4
[[ 0.03125  0.03125  0.03125  0.03125]
 [ 0.0625   0.125    0.03125  0.125  ]
 [ 0.125    0.25     0.5      1.     ]
 [-1.       0.5      1.       1.     ]] Iteration  5
[[ 0.03125   0.0625    0.015625  0.015625]
 [ 0.0625    0.125     0.0625    0.5     ]
 [ 0.125     0.25      0.5       1.      ]
 [-1.        0.5       1.        1.      ]] Iteration  6
[[ 0.03125  0.0625   0.03125  0.25

***Policy*** ***Iteration***

In [204]:
import numpy as np
# np.random.seed = 1
value_fun = np.zeros((grid_size,grid_size))
policy = np.random.randint(0,3,(4,4))
game = Game(grid_size,0.15)

episodes = 100
gamma = 0.95

def value_int(iterations,policy,grid_size):
  value_fun = np.zeros((grid_size,grid_size))
  value_fun[grid_size-1,grid_size-1] = 1
  value_fun[grid_size-1,0] = -1
  for i in range(iterations):
    v = np.zeros((grid_size,grid_size))
    v[grid_size-1,grid_size-1] = 1
    v[grid_size-1,0] = -1
    for r in range(grid_size):
      for c in range(grid_size):
        pos = [r,c]
        if game.is_end_state(pos):
          continue
        a = policy[r,c]
        new_pos,rew,d = game.take_action(pos,a)
        if not d:
          v[r,c] = rew + gamma*value_fun[new_pos[0],new_pos[1]]
        else:
          v[r,c] = rew
          #print(r,c,max_v)
    value_fun[:,:] = v[:,:]
#     print(value_fun,"Iteration ",i)
  return value_fun
  
def new_policy(policy,grid_size,value_fun):
  game2 = Game(4)
  new_p = np.array(policy)
  for r in range(grid_size):
    for c in range(grid_size):
      pos = [r,c]
      if game.is_end_state(pos):
        continue
      max_v = -2
      max_a = -1
      for a in range(n_action):
        new_pos,rew,d = game2.take_action(pos,a)
        if r == new_pos[0] and c == new_pos[1]:
          continue
        if max_v < value_fun[new_pos[0],new_pos[1]]:
          max_v = value_fun[new_pos[0],new_pos[1]]
          max_a = a
#         print(pos,a,new_pos,max_v,max_a)
      new_p[r,c] = max_a
  return new_p
  
for e in range(episodes):
  n_value = value_int(50,policy,grid_size)
  n_policy = new_policy(policy,grid_size,n_value)
  value_fun = np.array(n_value)
  policy = np.array(n_policy)
  print(policy, "Episode:", e)

[[2 1 3 3]
 [0 0 1 1]
 [2 2 1 1]
 [1 2 2 2]] Episode: 0
[[2 1 1 1]
 [1 1 1 1]
 [2 1 1 1]
 [1 2 2 2]] Episode: 1
[[1 1 2 1]
 [1 1 1 1]
 [2 1 1 1]
 [1 2 2 2]] Episode: 2
[[2 1 1 1]
 [1 1 1 1]
 [2 1 1 1]
 [1 2 2 2]] Episode: 3
[[2 1 2 1]
 [1 1 1 1]
 [2 1 1 1]
 [1 2 2 2]] Episode: 4
[[1 1 1 1]
 [1 1 1 1]
 [2 1 2 1]
 [1 0 2 2]] Episode: 5
[[1 2 1 1]
 [0 2 1 1]
 [0 2 1 1]
 [1 2 2 2]] Episode: 6
[[1 2 1 1]
 [2 1 1 1]
 [2 1 1 1]
 [1 2 2 2]] Episode: 7
[[2 1 1 1]
 [1 2 1 1]
 [0 1 1 1]
 [1 2 2 2]] Episode: 8
[[2 2 1 1]
 [2 1 1 0]
 [2 1 1 1]
 [1 2 2 2]] Episode: 9
[[1 1 1 3]
 [1 1 1 1]
 [2 1 1 1]
 [1 2 2 2]] Episode: 10
[[1 1 1 1]
 [1 1 1 1]
 [2 1 1 1]
 [1 2 2 2]] Episode: 11
[[1 1 1 1]
 [1 1 1 1]
 [2 1 1 1]
 [1 2 2 2]] Episode: 12
[[1 2 2 1]
 [1 1 1 1]
 [0 1 1 1]
 [1 2 2 2]] Episode: 13
[[2 1 1 1]
 [2 1 1 0]
 [2 1 1 1]
 [1 2 2 2]] Episode: 14
[[1 1 1 1]
 [1 0 2 1]
 [0 0 1 1]
 [1 2 2 2]] Episode: 15
[[1 2 1 1]
 [2 2 1 1]
 [0 1 1 1]
 [1 2 2 2]] Episode: 16
[[1 1 2 1]
 [2 1 1 1]
 [2 1 1 1]
 [1 2 2 